# Import modules

In [112]:
import json
from pprint import pprint
import glob
from datetime import datetime
import requests
import pandas as pd

# Self-define functions

In [30]:
## function for realtor api requests

def realtor_api_request(sort = "relevance", city = "Philadelphia", limit = "200", offset = "0", state_code = "PA"):
    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

    querystring = {
        "sort": sort,
        "city": city,
        "limit": limit,
        "offset": offset,
        "state_code": state_code
    }

    headers = {
        'x-rapidapi-host': "realtor.p.rapidapi.com",
        'x-rapidapi-key': "e2aef74c96msh10b708b155c5b67p1e6d19jsn4cb3a156a37d"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.json()

In [24]:
## convert response to pandas df

def process_response(response_json):
    '''
    This function is to convert each request result to a dataframe.
    
    1. create an empty list
    2. loop for each response and get details from key 'properties' 
    3. convert details to df
    4. append single df to list
    5. concat the list to one df
    
    '''
    
    # empty list
    convert_list=[]
    
    # loop through each 'properties'
    for col in response_json['properties']:
        
        # convert details to dataframe
        single_df = pd.DataFrame.from_dict(col, orient='index').T
        
        # append to list
        convert_list.append(single_df)
        
    # concat to a whole df, null for missing vals
    return pd.concat(convert_list, axis = 0, ignore_index=True, sort=False)

# Get data

In [44]:
## run for each offset
'''
There are 200 instances in each request. Restricted by Rapidapi
'''
def response_to_df(sort = "relevance", city = "Philadelphia", limit = "200", offset = "0", state_code = "PA"):
    property_for_sale = realtor_api_request(sort = sort, city = city, limit = limit, offset = offset, state_code = state_code)
    df_properties_for_sale_raw = process_response(response_json = property_for_sale)
    pd.set_option('display.max_columns', None)
    return df_properties_for_sale_raw


# Export response to csv file

In [58]:
## export each response to csv file
'''
Since each request could get 200 instances, after the each request, we increase the offset 200.
So on the second run, we can get data from 201 - 400, theoratically.
However, the system will keep updating the data so we may either get the data overlapped or miss some data.
We will deal with duplicated data later.

Currently, the loop will crash when the instances in the last run is less than 200. 
'''

start = datetime.now() ## to see how long will it finish the run
start = 0
for i in range(9467//200 + 1): ## the number for sale is 9467 since the last run
    response_to_df(offset = start).to_csv(f'test{i}.csv',encoding='utf-8', index=False)
    start += 200
    if i % 10 == 0:
        print(f'The {i}th Time')
        
end = datetime.now() -start

print(f'Total time is: {end}')

The 0th Time
The 10th Time
The 20th Time
The 30th Time
The 40th Time


JSONDecodeError: Extra data: line 1 column 5 (char 4)

# Data preprocessing

In [59]:
## import all csv to one df

path = r'E:\Projects\Drexel\DSCI 591 Capstone I\Projects\csv'
all_files = glob.glob(path + "/*.csv")

## empty list
file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)
    
combined_df = pd.concat(file_list, axis = 0, ignore_index=True)

combined_df

,property_id,listing_id,rdc_web_url,prop_type,prop_sub_type,address,branding,prop_status,price,baths_full,baths,beds,building_size,agents,office,last_update,client_display_flags,lead_forms,photo_count,thumbnail,page_no,rank,list_tracking,lot_size,mls,baths_half,virtual_tour,open_houses,plan_id,new_plan,quick_to_sell_days
0,M4046594895,2.922386e+09,https://www.realtor.com/realestateandhomes-det...,condo,duplex_triplex,"{'city': 'Philadelphia', 'line': '1516 N 62nd ...",{'listing_office': {'list_item': {'name': 'Arc...,for_sale,249900,3.0,3,6.0,"{'size': 1632, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '1291281',...","{'id': '3863c258f6142e2a2340175af3f64df0', 'na...",2020-10-13T17:54:05Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",9,https://ap.rdcpix.com/509b1ca1768ae6574e170ca7...,1,1,type|property|data|prop_id|4046594895|list_id|...,"{'size': 1842, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH944934', 'pla...",NaN,NaN,NaN,NaN,NaN,NaN
1,M3939384476,2.918257e+09,https://www.realtor.com/realestateandhomes-det...,condo,townhomes,"{'city': 'Philadelphia', 'line': '6102 Reedlan...",{'listing_office': {'list_item': {'name': 'Vih...,for_sale,116800,1.0,1,3.0,"{'size': 1092, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '347285', ...","{'id': 'a73ac4220ff5775a3f069bf0acab8e50', 'na...",2020-10-13T18:18:18Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",7,https://ap.rdcpix.com/f6ca6372b4cb434fd09d14b6...,1,2,type|property|data|prop_id|3939384476|list_id|...,"{'size': 992, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH911464', 'pla...",NaN,NaN,NaN,NaN,NaN,NaN
2,M4036371277,2.922385e+09,https://www.realtor.com/realestateandhomes-det...,condo,townhomes,"{'city': 'Philadelphia', 'line': '5703 N 13th ...",{'listing_office': {'list_item': {'name': 'Pre...,for_sale,215000,1.0,2,3.0,"{'size': 1360, 'units': 'sqft'}","[{'primary': True, 'photo': None, 'name': 'Kev...","{'id': '37e66a23ad2c2f122fe75b7df00955a5', 'na...",2020-10-13T17:24:20Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",35,https://ap.rdcpix.com/28d9bd1f5b0fdcb2de5b7aca...,1,3,type|property|data|prop_id|4036371277|list_id|...,"{'size': 1501, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH945010', 'pla...",1.0,NaN,NaN,NaN,NaN,NaN
3,M3553029343,2.922384e+09,https://www.realtor.com/realestateandhomes-det...,single_family,NaN,"{'city': 'Philadelphia', 'line': '1009 Rhawn S...",{'listing_office': {'list_item': {'name': 'Re/...,for_sale,394800,1.0,2,3.0,"{'size': 1856, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '4759', 'i...","{'id': 'd4965070728907fc00e98d31065c1b62', 'na...",2020-10-13T17:11:54Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",123,https://ap.rdcpix.com/792a419aa85110ba7780b21b...,1,4,type|property|data|prop_id|3553029343|list_id|...,"{'size': 15913, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH943096', 'pla...",1.0,NaN,NaN,NaN,NaN,NaN
4,M3649199107,2.922384e+09,https://www.realtor.com/realestateandhomes-det...,condo,townhomes,"{'city': 'Philadelphia', 'line': '3850 N Bouvi...",{'listing_office': {'list_item': {'name': 'Re/...,for_sale,130000,1.0,2,3.0,"{'size': 1180, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '391546', ...","{'id': 'a90ca2a01b749ef58fe768c91859baae', 'na...",2020-10-13T17:02:13Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",33,https://ap.rdcpix.com/bcbe5fa622b831b251d80654...,1,5,type|property|data|prop_id|3649199107|list_id|...,"{'size': 1240, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH918016', 'pla...",1.0,{'href': 'https://view.ricohtours.com/0701bde8...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

There are 9400 instances we achieved. However, there should be some duplicated instances. 

In [61]:
## remove duplicated instances
raw_data = combined_df.drop_duplicates(subset=['property_id'])
raw_data

,property_id,listing_id,rdc_web_url,prop_type,prop_sub_type,address,branding,prop_status,price,baths_full,baths,beds,building_size,agents,office,last_update,client_display_flags,lead_forms,photo_count,thumbnail,page_no,rank,list_tracking,lot_size,mls,baths_half,virtual_tour,open_houses,plan_id,new_plan,quick_to_sell_days
0,M4046594895,2.922386e+09,https://www.realtor.com/realestateandhomes-det...,condo,duplex_triplex,"{'city': 'Philadelphia', 'line': '1516 N 62nd ...",{'listing_office': {'list_item': {'name': 'Arc...,for_sale,249900,3.0,3,6.0,"{'size': 1632, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '1291281',...","{'id': '3863c258f6142e2a2340175af3f64df0', 'na...",2020-10-13T17:54:05Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",9,https://ap.rdcpix.com/509b1ca1768ae6574e170ca7...,1,1,type|property|data|prop_id|4046594895|list_id|...,"{'size': 1842, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH944934', 'pla...",NaN,NaN,NaN,NaN,NaN,NaN
1,M3939384476,2.918257e+09,https://www.realtor.com/realestateandhomes-det...,condo,townhomes,"{'city': 'Philadelphia', 'line': '6102 Reedlan...",{'listing_office': {'list_item': {'name': 'Vih...,for_sale,116800,1.0,1,3.0,"{'size': 1092, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '347285', ...","{'id': 'a73ac4220ff5775a3f069bf0acab8e50', 'na...",2020-10-13T18:18:18Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",7,https://ap.rdcpix.com/f6ca6372b4cb434fd09d14b6...,1,2,type|property|data|prop_id|3939384476|list_id|...,"{'size': 992, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH911464', 'pla...",NaN,NaN,NaN,NaN,NaN,NaN
2,M4036371277,2.922385e+09,https://www.realtor.com/realestateandhomes-det...,condo,townhomes,"{'city': 'Philadelphia', 'line': '5703 N 13th ...",{'listing_office': {'list_item': {'name': 'Pre...,for_sale,215000,1.0,2,3.0,"{'size': 1360, 'units': 'sqft'}","[{'primary': True, 'photo': None, 'name': 'Kev...","{'id': '37e66a23ad2c2f122fe75b7df00955a5', 'na...",2020-10-13T17:24:20Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",35,https://ap.rdcpix.com/28d9bd1f5b0fdcb2de5b7aca...,1,3,type|property|data|prop_id|4036371277|list_id|...,"{'size': 1501, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH945010', 'pla...",1.0,NaN,NaN,NaN,NaN,NaN
3,M3553029343,2.922384e+09,https://www.realtor.com/realestateandhomes-det...,single_family,NaN,"{'city': 'Philadelphia', 'line': '1009 Rhawn S...",{'listing_office': {'list_item': {'name': 'Re/...,for_sale,394800,1.0,2,3.0,"{'size': 1856, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '4759', 'i...","{'id': 'd4965070728907fc00e98d31065c1b62', 'na...",2020-10-13T17:11:54Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",123,https://ap.rdcpix.com/792a419aa85110ba7780b21b...,1,4,type|property|data|prop_id|3553029343|list_id|...,"{'size': 15913, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH943096', 'pla...",1.0,NaN,NaN,NaN,NaN,NaN
4,M3649199107,2.922384e+09,https://www.realtor.com/realestateandhomes-det...,condo,townhomes,"{'city': 'Philadelphia', 'line': '3850 N Bouvi...",{'listing_office': {'list_item': {'name': 'Re/...,for_sale,130000,1.0,2,3.0,"{'size': 1180, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '391546', ...","{'id': 'a90ca2a01b749ef58fe768c91859baae', 'na...",2020-10-13T17:02:13Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",33,https://ap.rdcpix.com/bcbe5fa622b831b251d80654...,1,5,type|property|data|prop_id|3649199107|list_id|...,"{'size': 1240, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH918016', 'pla...",1.0,{'href': 'https://view.ricohtours.com/0701bde8...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [139]:
## export to csv file
raw_data.to_csv('raw_data.csv', encoding='utf-8', index=False)

And now, we have 9364 out of 9467 instances.

In [135]:
## Remove unuseful cols
processed_data = raw_data.drop(columns=[
    'rdc_web_url',
    'office',
    'thumbnail',
    'list_tracking',
    'open_houses',
    'plan_id',
    'new_plan',
    'quick_to_sell_days',
    'virtual_tour',
    'client_display_flags',
    'lead_forms'
    
])
processed_data

,property_id,listing_id,prop_type,prop_sub_type,address,branding,prop_status,price,baths_full,baths,beds,building_size,agents,last_update,photo_count,page_no,rank,lot_size,mls,baths_half
0,M4046594895,2.922386e+09,condo,duplex_triplex,"{'city': 'Philadelphia', 'line': '1516 N 62nd ...",{'listing_office': {'list_item': {'name': 'Arc...,for_sale,249900,3.0,3,6.0,"{'size': 1632, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '1291281',...",2020-10-13T17:54:05Z,9,1,1,"{'size': 1842, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH944934', 'pla...",NaN
1,M3939384476,2.918257e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '6102 Reedlan...",{'listing_office': {'list_item': {'name': 'Vih...,for_sale,116800,1.0,1,3.0,"{'size': 1092, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '347285', ...",2020-10-13T18:18:18Z,7,1,2,"{'size': 992, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH911464', 'pla...",NaN
2,M4036371277,2.922385e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '5703 N 13th ...",{'listing_office': {'list_item': {'name': 'Pre...,for_sale,215000,1.0,2,3.0,"{'size': 1360, 'units': 'sqft'}","[{'primary': True, 'photo': None, 'name': 'Kev...",2020-10-13T17:24:20Z,35,1,3,"{'size': 1501, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH945010', 'pla...",1.0
3,M3553029343,2.922384e+09,single_family,NaN,"{'city': 'Philadelphia', 'line': '1009 Rhawn S...",{'listing_office': {'list_item': {'name': 'Re/...,for_sale,394800,1.0,2,3.0,"{'size': 1856, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '4759', 'i...",2020-10-13T17:11:54Z,123,1,4,"{'size': 15913, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH943096', 'pla...",1.0
4,M3649199107,2.922384e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '3850 N Bouvi...",{'listing_office': {'list_item': {'name': 'Re/...,for_sale,130000,1.0,2,3.0,"{'size': 1180, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '391546', ...",2020-10-13T17:02:13Z,33,1,5,"{'size': 1240, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH918016', 'pla...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9395,M3125042253,2.921793e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '1115 W Jeffe...",{'listing_office': {'list_item': {'name': 'Can...,for_sale,259900,1.0,1,3.0,"{'size': 1600, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '3271101',...",2020-09-26T20:23:59Z,8,47,18,"{'size': 1620, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH939452', 'pla...",NaN
9396,M3059705456,2.921793e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '1306 Robbins...",{'listing_office': {'list_item': {'name': 'RE/...,for_sale,165000,1.0,2,3.0,"{'size': 1146, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '3068413',...",2020-09-27T14:02:02Z,13,47,19,"{'size': 1260, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH939428', 'pla...",1.0
9397,M4077496585,2.921792e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '2318 Aspen S...",{'listing_office': {'list_item': {'name': 'BHH...,for_sale,685000,2.0,2,3.0,"{'size': 1988, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '198228', ...",2020-10-04T18:50:45Z,33,47,20,"{'size': 1120, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH938804', 'pla...",NaN
9398,M4371129467,2.921792e+09,condo,townhomes,"{'city': 'Philadelphia', 'line': '2019 Kimball...",{'listing_office': {'list_item': {'name': 'Com...,for_sale,399000,1.0,1,2.0,"{'size': 900, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '1736818',...",2020-09-26T19:05:20Z,20,47,21,"{'size': 720, 'units': 'sqft'}","{'name': 'BrightMLS', 'id': 'PAPH939440', 'pla...",NaN


In [138]:
## Export to csv file

processed_data.to_csv('processed_data.csv',encoding='utf-8', index=False)

There are some cols have details such `address`, `branding`, `agents`, `lot_size`, and `mls`. We need to discuss what features should we extract from them.